In [99]:
from scipy.fftpack import fft
from scipy.stats import linregress
import numpy as np

def compute_total_power(time_series):
    time_series = time_series.to_numpy()  # 👈 convert to numpy array
    n = len(time_series)
    fft_vals = fft(time_series)
    power_spectrum = np.abs(fft_vals[:n // 2])**2
    total_power = np.sum(power_spectrum)
    return total_power

def compute_power_law_slope(time_series, fs=1.0):
    time_series = time_series.to_numpy()  # 👈 convert to numpy array
    n = len(time_series)
    freqs = np.fft.fftfreq(n, d=1/fs)[:n // 2]
    fft_vals = fft(time_series)
    power = np.abs(fft_vals[:n // 2]) ** 2

    valid = freqs > 0
    log_freqs = np.log10(freqs[valid])
    log_power = np.log10(power[valid])

    slope, _, _, _, _ = linregress(log_freqs, log_power)
    return slope


In [89]:
import os

In [102]:
root_dir = "/scratch/data/90000189/BIDS/derivatives/fmriprep/def-plus/ts/scha17-1"
data_no_gsr_dir = "/scratch/data/90000189/BIDS/derivatives/fmriprep/def-plus/ts/scha17-1/simple-gsr_lp-0.09_fwhm-6"
data_with_gsr_dir = "/scratch/data/90000189/BIDS/derivatives/fmriprep/def-plus/ts/scha17-1/simple_lp-0.09_fwhm-6"

In [103]:
sessions = ['mec01', 'mec02', 'mec03', 'mec04', 'mec05', 'msl01', 'msl02', 'msl03', 'msl04', 'msl05', 'msl06']

In [104]:
os.path.isdir(root_dir)
os.listdir(root_dir)[:]


['simple-gsr_lp-0.09_fwhm-6', 'simple_lp-0.09_fwhm-6']

In [105]:
mrs_data_dir = "/data_downloads/MRS-data_from-benali"
os.path.isdir(mrs_data_dir)
os.listdir(mrs_data_dir)[:]

['PCC_new.mat', 'SD.mat', '__MACOSX', 'SMA_new.mat']

In [106]:
import scipy.io as sio
pcc_mrs_mat = sio.loadmat(os.path.join(mrs_data_dir,'PCC_new.mat'))
pcc_mrs = pcc_mrs_mat['PCC']

sma_mrs_mat = sio.loadmat(os.path.join(mrs_data_dir,'SMA_new.mat'))
sma_mrs = sma_mrs_mat['SMA']

pcc_mrs

array([[15.29 , 15.55 , 15.62 , 16.51 , 15.88 , 14.98 , 15.46 , 16.1  ,
        15.03 , 15.05 , 15.97 ],
       [15.43 , 14.   , 13.5  , 13.62 , 11.92 ,  9.127, 10.75 , 11.03 ,
           nan,  9.414,  7.814],
       [ 4.799, 14.6  , 11.11 , 13.78 , 13.25 , 14.64 , 13.27 , 14.93 ,
        13.42 , 14.1  , 15.61 ],
       [ 8.984, 10.63 , 13.62 ,    nan,    nan,  5.614, 14.38 , 14.42 ,
        14.83 , 15.02 , 15.33 ],
       [16.09 , 15.25 , 15.41 , 15.5  , 14.49 , 14.89 , 15.19 , 14.15 ,
        15.13 , 15.55 , 14.07 ],
       [12.32 , 14.76 , 13.98 , 13.91 , 12.51 , 11.2  , 14.31 , 13.58 ,
        15.14 , 14.5  , 12.17 ]])

In [107]:
file_path = os.path.join(data_no_gsr_dir, 'labels.txt')  

try:
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    processed_lines = [line.strip() for line in lines]
    

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")
    
print(len(processed_lines))

100


In [108]:
target_rois =target_rois = [
    "17Networks_LH_DefaultA_pCunPCC_1",
    "17Networks_RH_DefaultA_pCunPCC_1",
    "17Networks_LH_SomMotA_1",
    "17Networks_LH_SomMotA_2",
    "17Networks_RH_SomMotA_1",
    "17Networks_RH_SomMotA_2",
    "17Networks_RH_SomMotA_3",
    "17Networks_RH_SomMotA_4",
    "17Networks_LH_DorsAttnB_PostC_1",
    "17Networks_LH_DorsAttnB_PostC_2",
    "17Networks_LH_DorsAttnB_PostC_3",
    "17Networks_RH_DorsAttnB_PostC_1",
    "17Networks_RH_DorsAttnB_PostC_2"
]


target_rois_pcc= [
    "17Networks_LH_DefaultA_pCunPCC_1",
    "17Networks_RH_DefaultA_pCunPCC_1",
]

target_rois_sma =[
    "17Networks_LH_SomMotA_1",
    "17Networks_LH_SomMotA_2",
    "17Networks_RH_SomMotA_1",
    "17Networks_RH_SomMotA_2",
    "17Networks_RH_SomMotA_3",
    "17Networks_RH_SomMotA_4",
]

In [111]:
results=[]
for path in [(data_no_gsr_dir,'no gsr'),(data_with_gsr_dir,'with gsr')]:
    data_dir = path[0]
    for sub in range (2,8):
        sub_id = f"sub-{sub:02d}"
        for sess in sessions:
            filename = f"{sub_id}_ses-{sess}.csv"
            file_path = os.path.join(data_dir, filename)

            df = pd.read_csv(file_path)
            df.columns = processed_lines

            roi_df = df[target_rois]
            for roi in roi_df.columns:
                time_points = roi_df[roi]
                mean_val = np.mean(time_points)
                var_val = np.var(time_points)
                total_power = compute_total_power(time_points)
                slope = compute_power_law_slope(time_points)

                pcc_mrs_val = pcc_mrs[sub-2,sessions.index(sess)]
                sma_mrs_val = sma_mrs[sub-2,sessions.index(sess)]

                results.append({
                    "subject": sub_id,
                    "session": sess,
                    "roi": roi,
                    "gsr": path[1],
                    "mean": mean_val,
                    "variance": var_val,
                    "total power": total_power,
                    "power law slope": slope,
                    "pcc mrs": pcc_mrs_val,
                    "sma mrs": sma_mrs_val
                })


In [120]:
import pandas as pd
df_results = pd.DataFrame(results)
df_results.to_csv("fmri_features_with_mrs.csv", index=False)
df_results.head()

,subject,session,roi,gsr,mean,variance,total power,power law slope,pcc mrs,sma mrs
0,sub-02,mec01,17Networks_LH_DefaultA_pCunPCC_1,no gsr,1384.084756,11.052421,2.098856e+11,-2.357474,15.29,11.45
1,sub-02,mec01,17Networks_RH_DefaultA_pCunPCC_1,no gsr,1342.522984,13.479230,1.974700e+11,-3.906936,15.29,11.45
2,sub-02,mec01,17Networks_LH_SomMotA_1,no gsr,1528.789775,5.029322,2.560660e+11,-2.895188,15.29,11.45
3,sub-02,mec01,17Networks_LH_SomMotA_2,no gsr,1581.240097,2.665821,2.739377e+11,-1.724240,15.29,11.45
4,sub-02,mec01,17Networks_RH_SomMotA_1,no gsr,2005.650420,6.425380,4.407241e+11,-3.818444,15.29,11.45
